# Part 04 - Errors (Fantastic exceptions and where to find them) and File I/O

# Exceptions/Errors

As you're new programmers, you will spend most of your time handling errors. Sometimes, you even want to make errors of your own. Fortunately, you can teach python to handle errors or make them for you. Let's deliberately try something stupid and see what happens

In [9]:
import math
#logarithms of negative numbers are not valid
math.log(-1) 

ValueError: math domain error

Here we see something called `ValueError`, which has a `math domain error` message associated with it. Where did this come from?

When bad values are passed to a function, it can't return a value as that might be misinterpreted as the answer. Instead, we raise an exception. Exceptions are not return values but instead bubble up the stack until they are caught or they come out of the main scope where they cause python to exit.

The stack is all the functions that python has had to enter to get to the current line of execution. Going up the stack, python looks for anything that might catch the exception. Lets see how that's done, 

In [ ]:
try:
    math.log(-1)
except Exception as e:
    print(e)
    print(type(e))
    print(repr(e))

If any commands within a `try` block raise an exception, and the exception rises up to the `try` block then it will be caught and the code in the `except:` block is run. Above, we just print the exception, the type of the exception, and we use the `repr` command which returns the a python command which will generate the object passed as an argument.

The `try...catch` block above catches all exceptions (as all exceptions derive from the `Exception` class). We can actually catch specfic exceptions and deal with them in a special way,

In [ ]:
try:
    math.log(-1)
except ValueError as e:
    print("Bad value used")
except Exception as e:
    print("Some other error happened!", e)

How do we raise exceptions ourselves? We just write raise, then use an exception type:

In [ ]:
raise RuntimeError("My own exception")

Lets look how you might use it to improve our prime number function,

In [ ]:
import math
def isPrime(n):
    # Perform sanity checks on the input
    if isinstance(n, float):
        if n.is_integer():
            n = int(n)
        else:
            raise ValueError("isPrime cannot accept fractional floating values")
    
    if not isinstance(n, int):
        raise ValueError("isPrime can only accept integer arguments")
        
    for div in range(2,int(math.sqrt(n))):
        if n % div == 0:
            return False
    return True

print(2**31-1, isPrime(2**31-1))
print(3.0, isPrime(3.0))

#Errors which raise exceptions!
isPrime(2.2)
isPrime("Not an integer")

We now get nice error messages when we abuse our `isPrime` function.

## File I/O

Now we know about exceptions, we can actually take a look at file Input/Output.

Opening a file is very easy, and once its open we can even write to it:

In [ ]:
# Open a file called myfile.txt for (w)riting
with open('myfile.txt', 'w') as output:
    print("The first line in the file", file=output)
    print("The second line in the file", file=output)

The `with` block is a way of making sure that the next bit of code ONLY happens if the file can be opened. Also, it guarantees that if anything goes wrong inside the `with` block (and an exception is raised), the file will be closed before the exception makes it past the `with` (although the file will be closed anyway if python exits during garbage collection). 

We could just write `output = open('myfile.txt')` and it would work, but this is better at handling errors.

Now we have a file written out, how do we read it?

In [ ]:
# Now its open for (r)eading
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(next(lines))
    except StopIteration:
        print('EOF!')

The `while` statement repeats the block as long as the expression to its right is `True`. Here, the expression is `True` so the block is repeated indefinitely. The only way that block can end is if an exception is raised. Fortunately, when the file runs out of lines, the `next()` function will throw a `StopIteration` exception, which we catch to say End Of File (EOF)! This is a case where we MUST use exceptions to achieve our goals.

If you notice the results from the above code, we have extra lines appearing, where do these come from? Lets take another look but this time using `repr`

In [ ]:
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(repr(next(lines)))
    except StopIteration:
        print('EOF!')

Here we can see that each line is being read from the file including its terminating newline character. When we print this, both the terminating newline `\n` and one automatically added by `print` is being output resulting in a double line break. We can fix this by telling print not to end the line that way:

In [ ]:
with open('myfile.txt', 'r') as lines:
    try:
        while True:
            print(next(lines), end='')
    except StopIteration:
        print('EOF!')

## Better file IO with pickle

Often you don't want to save text, you want to save the array or list you've been working on. Python has a cool library for that called pickle. 

In [ ]:
import pickle
favorite_color = { "lion": "yellow", "kitty": "red" }
pickle.dump( favorite_color, open("save.p", "wb"))

We've been fast and loose here, not using a `with`, but sometimes we like to program dangerously. You might notice the `'b'` added to the `open` command, this opens the file in binary mode, which is very space efficient but not readable by humans.

Now, if we want to reload this, we just do the following:

In [10]:
favorite_color_loaded = pickle.load( open( "save.p", "rb" ) )
print(favorite_color_loaded)

{'kitty': 'red', 'lion': 'yellow'}
